In [1]:
# 按照距离序列划分序列

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import points_in_box
from scipy.spatial.transform import Rotation as R
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import LidarPointCloud 
from nuscenes.utils.geometry_utils import transform_matrix
from PIL import Image 
import numpy as np 
import shutil, os

raw_data_dir = '/home/gs/workspace/datasets/nuScenes/trainval/'
save_data_dir = '/home/gs/workspace/datasets/nuScenes/seq_bev_dataset/'

#dataroot='/home/gs/Datasets/nuScenes/mini'
dataroot='/home/gs/workspace/datasets/nuScenes/trainval/'
#map_name='singapore-onenorth'

nusc = NuScenes(version='v1.0-trainval', dataroot=dataroot, verbose=True)  # 'v1.0-trainval' 'v1.0-mini'





Loading NuScenes tables for version v1.0-trainval...
Loading nuScenes-lidarseg...
Loading nuScenes-panoptic...
32 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
34149 lidarseg,
34149 panoptic,
Done loading in 42.137 seconds.
Reverse indexing ...
Done reverse indexing in 4.4 seconds.
0


NameError: name 'my_scene' is not defined

In [35]:
import csv

raw_data_dir = '/home/gs/workspace/datasets/nuScenes/trainval/'
save_data_dir = '/home/gs/workspace/datasets/nuScenes/seq_bev_dataset/'


# check and create folders
def check_and_create_dirs(paths):
    for path in paths:        
        if not os.path.isdir(path):
            print('%s does not exist, create the folder' %path)
            os.makedirs(path)
        else:
            print('%s exist, will delete and create the folder' %path)
            shutil.rmtree(path) 
            os.makedirs(path)
            
def get_dist(cur_pose, next_pose, CAM):
    # cur_camera_data = nusc.get('sample_data', current_sample['data'][CAM])
    # cur_pose = nusc.get('ego_pose', cur_camera_data['ego_pose_token'])
    # next_camera_data = nusc.get('sample_data', next_sample['data'][CAM])
    # next_pose = nusc.get('ego_pose', next_camera_data['ego_pose_token'])
    cur_t = np.array(cur_pose['translation'].copy())
    next_t = np.array(next_pose['translation'].copy())
    dist = np.sqrt(np.sum(np.square(cur_t-next_t)))
    return dist

def get_angle(cur_pose, next_pose, CAM):
    cur_p = cur_pose['rotation'].copy()   # nuscenes的四元数顺序wxyz
    next_p = next_pose['rotation'].copy()
    
    cur_p.insert(len(cur_p), cur_p[0])  # 交换四元数顺序，以便用scipy包进行欧拉角变换
    cur_p.remove(cur_p[0])
    next_p.insert(len(next_p), next_p[0])
    next_p.remove(next_p[0])
    
    cur_r = R.from_quat(cur_p)  # 转成旋转矩阵
    next_r = R.from_quat(next_p)
    cur_euler = cur_r.as_euler('zxy', degrees=True)  # 转成欧拉角
    next_euler = next_r.as_euler('zxy', degrees=True)
    rotation = abs((cur_euler-next_euler)[0])
    return rotation
    
    
#for scene_id in range(len(nusc.scene)):
for scene_id in range(2):
    print(scene_id)
    scene = nusc.scene[scene_id]
    #print(scene)
    
    # TODO
    save_root = os.path.join(save_data_dir, scene['name'])
    save_image_path = os.path.join(save_root, 'cam_front')
    #print(">>>>>>>>>>>>save_image_path: ", save_image_path)
    save_BEVgt_path = os.path.join(save_root, 'BEV_gt')
    #print(">>>>>>>>>>>>save_gt_path: ", save_gt_path)
    save_roadLayoutGT_path = os.path.join(save_root, 'roadLayout_gt')
    paths_list = [save_root, save_image_path, save_BEVgt_path, save_roadLayoutGT_path]
    check_and_create_dirs(paths_list)  # create folders
    scene_file_path = save_root + '/input.csv'  # create csv file
    with open(scene_file_path, 'w') as f:
        csv_write = csv.writer(f)
    
    CAM = 'CAM_FRONT'
    
    sample_num = 0
    
    first_sample_token = scene['first_sample_token']
    current_sample = nusc.get('sample', first_sample_token)
    #cur_sample = current_sample
    
    while(True):
        print('Scene No. {}/{}, Sample No. {}/{}: '.format(scene_id+1, len(nusc.scene), sample_num+1, scene['nbr_samples']))
        
        ################ Camera Data #################
        camera_data = nusc.get('sample_data', current_sample['data'][CAM])
        image_filename = camera_data['filename']  # 存图时的图像位置img_front = Image.open(os.path.join(raw_data_dir, image_filename))
        sample_token = camera_data['sample_token']  # sample_token作为全局索引
        save_image_name = '{:0>3}'.format(sample_num)+'_'+sample_token+'.png' 
        print('save image name: ', save_image_name)
        img_front = Image.open(os.path.join(raw_data_dir, image_filename))
        img_front_resized = img_front.resize((512,256),Image.LANCZOS) # 相比于原图缩小一倍
        ego_pose = nusc.get('ego_pose', camera_data['ego_pose_token'])
        cur_sample = current_sample
        #sample_token_row = []
        num_seq = 3
        
        # 保存原始图像
        img_front_resized.save(os.path.join(save_image_path,'{:0>3}'.format(sample_num)+'_'+sample_token+'.png'))
        
        # 向csv文件中记录前后3帧的sample_token
        for num_frame in range(num_seq-1):   # 前后3帧
            dist = 0
            angle = 0
            num_next = 0
            print('*'*60)
            while(True):
                
                cur_sample_token = cur_sample['token']
                print('cur_sample_token: {}'.format(cur_sample_token))
                cur_camera_data = nusc.get('sample_data', cur_sample['data'][CAM])
                cur_pose = nusc.get('ego_pose', cur_camera_data['ego_pose_token'])

                if num_next == 0:
                    next_sample_token = cur_sample['next']
                if next_sample_token == '':
                    break
                print('next sample token: {}'.format(next_sample_token))
                next_sample = nusc.get('sample', next_sample_token)
                next_camera_data = nusc.get('sample_data', next_sample['data'][CAM])
                next_pose = nusc.get('ego_pose', next_camera_data['ego_pose_token'])
                
                if dist < 10 and angle < 45:
                    dist = get_dist(cur_pose, next_pose, CAM)
                    angle = get_angle(cur_pose, next_pose, CAM)
                    print('dist: {}, angle: {}'.format(dist, angle))
                    #next_sample_token_old = 
                    #next_sample_token = nusc.get('sample', next_sample['next'])['next']
                    next_sample_token = next_sample['next']
                    print('new next sample token: {}'.format(next_sample_token))
                else:
                    # TODO record next_sample_token into the txt file
                    print('***fond great dist or angle***')
                    print('cur_sample_token: {}; next_sample_token:{}'.format(cur_sample_token, next_sample['prev']))  # next_sample已经指向下一个sample，为得到满足条件的sample,所以用next_sample['prev']
                    if num_frame == 0:
                        sample_token_row = [cur_sample_token, next_sample['prev']]
                    else:
                        sample_token_row.append(next_sample['prev'])
                    if len(sample_token_row)==num_seq:
                        with open(scene_file_path, 'a+') as f:
                            csv_write = csv.writer(f)
                            print("<<<<<<<<<<<<<sample_token_row: ", sample_token_row)
                            csv_write.writerow(sample_token_row)
              
                    cur_sample = nusc.get('sample', next_sample['prev'])
                    break
                num_next += 1
    
        # TODO generate ground truth
        
        
        sample_num += 1
        if (current_sample['next'] == ''):
            print('!There is no more next!')
            break
        current_sample = nusc.get('sample', current_sample['next'])

0
/home/gs/workspace/datasets/nuScenes/seq_bev_dataset/scene-0001 exist, will delete and create the folder
/home/gs/workspace/datasets/nuScenes/seq_bev_dataset/scene-0001/cam_front does not exist, create the folder
/home/gs/workspace/datasets/nuScenes/seq_bev_dataset/scene-0001/BEV_gt does not exist, create the folder
/home/gs/workspace/datasets/nuScenes/seq_bev_dataset/scene-0001/roadLayout_gt does not exist, create the folder
Scene No. 1/850, Sample No. 1/40: 
save image name:  000_e93e98b63d3b40209056d129dc53ceee.png
************************************************************
cur_sample_token: e93e98b63d3b40209056d129dc53ceee
next sample token: 14d5adfe50bb4445bc3aa5fe607691a8
dist: 2.094743875064927, angle: 8.945690003474326
new next sample token: ae4e0c3aa3f24c91aab599e8b54e9264
cur_sample_token: e93e98b63d3b40209056d129dc53ceee
next sample token: ae4e0c3aa3f24c91aab599e8b54e9264
dist: 4.052298042394314, angle: 19.526044526500783
new next sample token: 8de7ec06e1ac48c689c4d24d6cc

/tmp/ipykernel_1070640/3202100755.py:82: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img_front_resized = img_front.resize((512,256),Image.LANCZOS) # 相比于原图缩小一倍


************************************************************
cur_sample_token: 8de7ec06e1ac48c689c4d24d6cc64fd7
next sample token: ba94cb79ebc74614bc2442185cb53c26
dist: 2.166566549747985, angle: 11.40715309801162
new next sample token: 4dfab70f0b3f405bbf17f7c534d2dd91
cur_sample_token: 8de7ec06e1ac48c689c4d24d6cc64fd7
next sample token: 4dfab70f0b3f405bbf17f7c534d2dd91
dist: 4.849843107188075, angle: 22.530841817059596
new next sample token: 140c678882194ae69e47136c81168b6d
cur_sample_token: 8de7ec06e1ac48c689c4d24d6cc64fd7
next sample token: 140c678882194ae69e47136c81168b6d
dist: 7.356019231184885, angle: 26.444105142595944
new next sample token: b8002d4effb540c0b1bb70addf423f32
cur_sample_token: 8de7ec06e1ac48c689c4d24d6cc64fd7
next sample token: b8002d4effb540c0b1bb70addf423f32
dist: 9.528891235425208, angle: 27.213665605921406
new next sample token: 1e19d0a5189b46f4b62aa47508f2983e
cur_sample_token: 8de7ec06e1ac48c689c4d24d6cc64fd7
next sample token: 1e19d0a5189b46f4b62aa47508f298

************************************************************
cur_sample_token: b8002d4effb540c0b1bb70addf423f32
next sample token: 1e19d0a5189b46f4b62aa47508f2983e
dist: 2.8038669745749436, angle: 0.18256444412932638
new next sample token: b7f4a3b32d654cc2b69c1c260a5f34a0
cur_sample_token: b8002d4effb540c0b1bb70addf423f32
next sample token: b7f4a3b32d654cc2b69c1c260a5f34a0
dist: 5.745538592126085, angle: 0.140528135826969
new next sample token: 917a76cd5f564387bcacc22537010630
cur_sample_token: b8002d4effb540c0b1bb70addf423f32
next sample token: 917a76cd5f564387bcacc22537010630
dist: 8.722294419190652, angle: 0.27194085268860135
new next sample token: c0cd427afacb4c2bb974461eec1dbd79
cur_sample_token: b8002d4effb540c0b1bb70addf423f32
next sample token: c0cd427afacb4c2bb974461eec1dbd79
dist: 11.658277454584907, angle: 0.24605612392423382
new next sample token: c36eb85918a84a788e236f5c9eef2b05
cur_sample_token: b8002d4effb540c0b1bb70addf423f32
next sample token: c36eb85918a84a788e236f5c9

************************************************************
cur_sample_token: c36eb85918a84a788e236f5c9eef2b05
next sample token: 5d054297a2bf43638a1d37cbb2ea2365
dist: 2.670286023308375, angle: 0.004421577938330756
new next sample token: fcd37961086342abaf431243971cbc9b
cur_sample_token: c36eb85918a84a788e236f5c9eef2b05
next sample token: fcd37961086342abaf431243971cbc9b
dist: 5.024720083257147, angle: 0.23102522064479558
new next sample token: 6ec48adcac3b407ca77de01d2c600bf4
cur_sample_token: c36eb85918a84a788e236f5c9eef2b05
next sample token: 6ec48adcac3b407ca77de01d2c600bf4
dist: 7.50476907294745, angle: 0.8469853308548068
new next sample token: 7ff1f84a3c184434a366282773ca9572
cur_sample_token: c36eb85918a84a788e236f5c9eef2b05
next sample token: 7ff1f84a3c184434a366282773ca9572
dist: 9.786096816191705, angle: 2.2160598278089196
new next sample token: 9253e18a84404e25b1eb4433905ad579
cur_sample_token: c36eb85918a84a788e236f5c9eef2b05
next sample token: 9253e18a84404e25b1eb4433905

...

************************************************************
cur_sample_token: eac9992171c446dabf9d738c23a96cde
next sample token: fa88c561566341479ad9bfcd557b096b
dist: 3.166147374482849, angle: 0.36798302153307816
new next sample token: 3623e20bb2e647a280cbb6d8a0b50874
cur_sample_token: eac9992171c446dabf9d738c23a96cde
next sample token: 3623e20bb2e647a280cbb6d8a0b50874
dist: 6.34528342287566, angle: 0.2600389676626378
new next sample token: 7616699455044566a7bf555d476f5799
cur_sample_token: eac9992171c446dabf9d738c23a96cde
next sample token: 7616699455044566a7bf555d476f5799
dist: 9.843702500829309, angle: 0.03505048824899859
new next sample token: 6b72920576b04269ab1b929599e8dc79
cur_sample_token: eac9992171c446dabf9d738c23a96cde
next sample token: 6b72920576b04269ab1b929599e8dc79
dist: 13.63898410413597, angle: 0.3400699437885635
new next sample token: 6b615c1f0be2421dbb6337ea8d5ee04e
cur_sample_token: eac9992171c446dabf9d738c23a96cde
next sample token: 6b615c1f0be2421dbb6337ea8d5e

************************************************************
cur_sample_token: 6b615c1f0be2421dbb6337ea8d5ee04e
next sample token: bb4f4d9a36754178a62d5dfbd55998b9
dist: 3.917466597284945, angle: 0.24160787901014658
new next sample token: d6515448fc1e4af98571ae1663b33efe
cur_sample_token: 6b615c1f0be2421dbb6337ea8d5ee04e
next sample token: d6515448fc1e4af98571ae1663b33efe
dist: 7.303031344527131, angle: 0.5864287062044156
new next sample token: b143df90cc734dd780de782bdd261ab1
cur_sample_token: 6b615c1f0be2421dbb6337ea8d5ee04e
next sample token: b143df90cc734dd780de782bdd261ab1
dist: 11.014818204383706, angle: 1.166924856483675
new next sample token: 4dd38dcd4e8549d6a37938285f886117
cur_sample_token: 6b615c1f0be2421dbb6337ea8d5ee04e
next sample token: 4dd38dcd4e8549d6a37938285f886117
***fond great dist or angle***
cur_sample_token: 6b615c1f0be2421dbb6337ea8d5ee04e; next_sample_token:b143df90cc734dd780de782bdd261ab1
************************************************************
cur_sample

************************************************************
cur_sample_token: 5b426b1475f746f7904f2bf1db929852
next sample token: 40a982ccf9564c6ea574f1d75f8a7dc0
dist: 3.6250835577138836, angle: 0.09314203528229825
new next sample token: 9d601e6912d04c77951c173f6012ee6f
cur_sample_token: 5b426b1475f746f7904f2bf1db929852
next sample token: 9d601e6912d04c77951c173f6012ee6f
dist: 7.384489645360931, angle: 0.43806560340043177
new next sample token: a05edf71b4b14052bb477918d0997c88
cur_sample_token: 5b426b1475f746f7904f2bf1db929852
next sample token: a05edf71b4b14052bb477918d0997c88
dist: 10.253524966010179, angle: 1.0899311109332785
new next sample token: cc0a7d86675248779f21b34ae891159c
cur_sample_token: 5b426b1475f746f7904f2bf1db929852
next sample token: cc0a7d86675248779f21b34ae891159c
***fond great dist or angle***
cur_sample_token: 5b426b1475f746f7904f2bf1db929852; next_sample_token:a05edf71b4b14052bb477918d0997c88
************************************************************
cur_sam

************************************************************
cur_sample_token: 5b89841bc40a436891223751749416b3
next sample token: d8a6a5c6a7c444a9aba383526e5e1161
dist: 2.0910319837176696, angle: 0.6460820840635364
new next sample token: a0c1b7a7cec84f6f8e2955ac370637ad
cur_sample_token: 5b89841bc40a436891223751749416b3
next sample token: a0c1b7a7cec84f6f8e2955ac370637ad
dist: 4.221165986490208, angle: 0.8903516195331633
new next sample token: a0197f52509f48feb1292b78c9188d1f
cur_sample_token: 5b89841bc40a436891223751749416b3
next sample token: a0197f52509f48feb1292b78c9188d1f
dist: 6.042681560157159, angle: 1.1556221074145299
new next sample token: 65282c155f014646868b295b7c6d6e52
cur_sample_token: 5b89841bc40a436891223751749416b3
next sample token: 65282c155f014646868b295b7c6d6e52
dist: 8.626543987079678, angle: 1.3293696261144845
new next sample token: d9b482ce80634d5a819aac651c1e31d3
cur_sample_token: 5b89841bc40a436891223751749416b3
next sample token: d9b482ce80634d5a819aac651c1e

************************************************************
cur_sample_token: d9b482ce80634d5a819aac651c1e31d3
next sample token: b0c254b0902d464798cc86878317ec06
dist: 3.211448141730561, angle: 0.17242755526223164
new next sample token: aadec83b3fa24b9c87651c3d2ac53035
cur_sample_token: d9b482ce80634d5a819aac651c1e31d3
next sample token: aadec83b3fa24b9c87651c3d2ac53035
dist: 6.853902202533492, angle: 0.1941976536805896
new next sample token: 36d4e4bf966e467bbee01dd6e63d5cce
cur_sample_token: d9b482ce80634d5a819aac651c1e31d3
next sample token: 36d4e4bf966e467bbee01dd6e63d5cce
dist: 10.746898738973346, angle: 0.36561409870338935
new next sample token: 8df9376254584396bdfaa521242b4757
cur_sample_token: d9b482ce80634d5a819aac651c1e31d3
next sample token: 8df9376254584396bdfaa521242b4757
***fond great dist or angle***
cur_sample_token: d9b482ce80634d5a819aac651c1e31d3; next_sample_token:36d4e4bf966e467bbee01dd6e63d5cce
************************************************************
cur_samp